In [ ]:
#!/usr/bin/env julia
# ============================================================================
#  Five-asset min-basket call
#    1. tf-quant-finance Monte-Carlo oracle (via PyCall)
#    2. Tensor-cross interpolation  → initial TT
#    3. L-BFGS refit (Zygote + OptimKit) → refined TT
#
#   Julia ≥ 1.10  with packages:
#       PyCall Statistics LinearAlgebra Random Distributions
#       TensorCrossInterpolation Zygote OptimKit JLD2
#       (Python side: tensorflow, tf-quant-finance)
# ============================================================================

using PyCall, Statistics, LinearAlgebra, Random, Distributions
using TensorCrossInterpolation
import TensorCrossInterpolation: CachedFunction, optfirstpivot, crossinterpolate2
using Zygote, OptimKit, JLD2

# ---------------- Python (tf-quant-finance) ----------------
const np = pyimport("numpy")
@pyimport tf_quant_finance as tff
@pyimport tensorflow as tf


# ═══════════════════════════════════════════════════════════
#  Utility functions
# ═══════════════════════════════════════════════════════════

chebyshev_lobatto_nodes(n, a, b) = begin
    xs = [cos(π*(n-1-i)/(n-1)) for i in 0:n-1]
    return 0.5*(b-a).*(xs .+ 1) .+ a, xs
end

function calculate_option_price(
    T,r,K,d, idx, σg,S0g, ρ, nPath, seed)

    vols = σg[idx[1:2:end]]
    S0s  = S0g[idx[2:2:end]]

    np.random.seed(seed);  tf.random.set_seed(seed)

    gbm = tff.models.MultivariateGeometricBrownianMotion(
        dim=d, means=fill(r,d), volatilities=vols, corr_matrix=ρ)

    paths = gbm.sample_paths(
        times=[T], initial_state=S0s,
        random_type=tff.math.random.RandomType.PSEUDO,
        seed=seed, num_samples=nPath).numpy()

    payoff = max.(minimum(paths[:,1,:], dims=2) .- K, 0.0)
    disc   = exp(-r*T)
    return disc*mean(payoff), disc*std(payoff)
end


function monte_carlo_error_stats(reps, T,r,K,d, σg,S0g, ρ, nPath, seed)
    Random.seed!(seed)
    errs = Float64[]
    for _ in 1:reps
        idx = [rand(1:length(σg)) for _ in 1:2d]
        _, s = calculate_option_price(T,r,K,d, idx,σg,S0g,ρ, nPath, seed)
        push!(errs, s/sqrt(nPath))
    end
    mean(errs), maximum(errs)
end


function tci_oneshot(f, dims, firstpivot, tol; max_threads=4)
    BLAS.set_num_threads(max_threads)
    # small local search
    for _ in 1:100
        p = optfirstpivot(f, dims, firstpivot)
        firstpivot = abs(f(p)) > abs(f(firstpivot)) ? p : firstpivot
    end
    crossinterpolate2(
        Float64, f, dims, [firstpivot];
        tolerance=tol, maxiter=6, pivotsearch=:rook,
        maxbonddim=50, verbosity=1, loginterval=1)
end


# ═══════════════════════════════════════════════════════════
#  Main workflow
# ═══════════════════════════════════════════════════════════

function main()
    # -------- parameters ----------
    T,r,K     = 1.0, 0.01, 100.0
    d         = 5
    nPath     = 10^6          # ★ extremely small for quick run
    seed      = 1234
    num_grid  = 100
    Random.seed!(seed)

    # -------- grids ---------------
    σg,_  = chebyshev_lobatto_nodes(num_grid, 0.15, 0.25)
    S0g,_ = chebyshev_lobatto_nodes(num_grid, 90.0, 120.0)

    # -------- correlation ---------
    ρ = [ 1.0 0.303659 0.505532 0.719655 0.728832;
          0.303659 1.0 0.401077 0.515272 0.178203;
          0.505532 0.401077 1.0 0.132542 0.722767;
          0.719655 0.515272 0.132542 1.0 0.394723;
          0.728832 0.178203 0.722767 0.394723 1.0 ]

    # -------- baseline ------------
    base_idx = fill(num_grid, 2d)
    μ, σ = calculate_option_price(T,r,K,d, base_idx,σg,S0g,ρ, nPath, seed)
    @info "Baseline PV = $μ,  abs.err = $(σ/sqrt(nPath))"

    μe, maxe = monte_carlo_error_stats(100,T,r,K,d,σg,S0g,ρ,nPath,seed)
    @info "MC abs.err  mean=$μe  max=$maxe"

    # -------- TCI -----------------
    oracle(x) = Float64(calculate_option_price(
                            T,r,K,d, x, σg,S0g,ρ, nPath, seed)[1])  # ← 常に Float64

    dims, firstpivot = fill(num_grid, 2d), rand(1:num_grid, 2d)
    tt0, tci_err     = tci_oneshot(
                           CachedFunction{Float64}(oracle, dims),
                           dims, firstpivot, 1e-3)
    @info "TCI finished  |err|∞ = $(last(tci_err))"

    # -------- gradient refit ------
    φ          = CachedFunction{Float64}(oracle, dims)  # to get the dict
    itpl_dict  = φ.itpl_point_dict
    indexsets  = sort(collect(keys(itpl_dict)))
    vals       = [itpl_dict[I] for I in indexsets]

    tt0s  = TensorCrossInterpolation.TensorTrain(tt0)
    ttfit = TensorCrossInterpolation.TensorTrainFit{Float64}(indexsets, vals, tt0s)
    x0    = TensorCrossInterpolation.flatten(tt0s)

    loss_and_grad(x) = begin
        ℓ, (∇,) = Zygote.withgradient(ttfit, x)   # Zygote gives a tuple of 1 gradient
        return ℓ, ∇
    end

    xopt, fvals, gnorms, nit, _ =
        OptimKit.optimize(loss_and_grad, x0, OptimKit.LBFGS(maxiter=100, verbosity=1))

    @info "LBFGS done: iter=$nit  loss=$(last(fvals))  ‖∇‖=$(last(gnorms))"

    tt_opt = TensorCrossInterpolation.TensorTrain{Float64,3}(
                 TensorCrossInterpolation.to_tensors(ttfit, xopt))

    # -------- save ----------------
    @save "tt_tci_$(nPath)_raw.jld2" tt0
    @save "tt_tci_$(nPath)_opt.jld2" tt_opt
end


# ---------------------------------------------------------------------------
main()
# ---------------------------------------------------------------------------


┌ Info: Baseline PV = 0.922077512338028,  abs.err = 0.9220775123380279
└ @ Main /Users/sakurairihito/Documents/Research/post-doc/Tensor_FixedSeedMC/notebook/multi-asset/Europian_min_op/random_corr/all_gr100_opt/10^1/make_tt/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X20sZmlsZQ==.jl:109
┌ Info: MC abs.err  mean=0.0526798158402132  max=1.79919023427231
└ @ Main /Users/sakurairihito/Documents/Research/post-doc/Tensor_FixedSeedMC/notebook/multi-asset/Europian_min_op/random_corr/all_gr100_opt/10^1/make_tt/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X20sZmlsZQ==.jl:112


iteration = 1, rank = 16, error= 0.0, maxsamplevalue= 4.960692240374436, nglobalpivot=1
  Rejected 1 global pivots added in the previous iteration, errors are [0.0007366071086800652]
iteration = 2, rank = 36, error= 0.0, maxsamplevalue= 4.960692240374436, nglobalpivot=1
  Rejected 1 global pivots added in the previous iteration, errors are [0.0038122741491914275]
iteration = 3, rank = 52, error= 0.0, maxsamplevalue= 4.960692240374436, nglobalpivot=2


InterruptException: InterruptException:

In [7]:
# 満期・金利・行使価格など
T = 1.0               # 満期
r = 0.01              # 無リスク金利
K = 100.0             # 行使価格
d = 5                 # 原資産数
nPath = 10^1           # モンテカルロパス数
S0 = fill(100, d)     # 原資産の初期値ベクトル
seed = 1234

function chebyshev_lobatto_nodes(n::Int, a::Real, b::Real)
    # [-1,1] のチェビシェフ-ロバット節点 → [a,b] へ写像
    x_standard = [cos(π * (n - 1 - i) / (n - 1)) for i in 0:n-1]
    x_scaled = 0.5 * (b - a) .* (x_standard .+ 1) .+ a
    return x_scaled, x_standard
end

num_nodes = 100
σs, _ = chebyshev_lobatto_nodes(num_nodes, 0.15, 0.25)    # ボラティリティグリッド
stock0, _ = chebyshev_lobatto_nodes(num_nodes, 90, 120)   # 初期株価グリッド

corrMat = [
    1.0      0.303659  0.505532  0.719655  0.728832;
    0.303659 1.0      0.401077  0.515272  0.178203;
    0.505532 0.401077 1.0      0.132542  0.722767;
    0.719655 0.515272 0.132542 1.0      0.394723;
    0.728832 0.178203 0.722767 0.394723  1.0
]

function calculate_option_price(
    T::Float64, r::Float64, K::Float64, d::Int, S0::Vector{Int}, stock0::Vector{Float64},
    localindex_v::Vector{Int}, nPath::Int, σs::Vector{Float64}, corrMat
)::Tuple{Float64, Float64}
    # ボラリティ、初期株価の組み合わせを抽出
    vols = [σs[i] for i in localindex_v[1:2:2*d]]
    S0s = [stock0[i] for i in localindex_v[2:2:2*d]]

    np.random.seed(seed)  # numpyのシードを固定
    tf.random.set_seed(seed)  # tensorflowのシードを固定
    
    process = tff.models.MultivariateGeometricBrownianMotion(
        dim=d,
        means=fill(r, d),
        volatilities=vols,
        corr_matrix=corrMat)
    
    paths = process.sample_paths(
        times=[T],
        initial_state=S0s,
        random_type=tff.math.random.RandomType.PSEUDO,
        seed = seed,
        num_samples=nPath
    ).numpy()
    
    payoffs = max.(minimum(paths[:, 1, :], dims=2) .- K, 0.0)
    mean_pv = exp(-r * T) * mean(payoffs)
    std_pv = exp(-r * T) * std(payoffs)
    return (mean_pv, std_pv)
end


calculate_option_price (generic function with 1 method)

In [8]:
# struct to count number of function calls and sampling points
mutable struct WrapFuncFloat <: Function
    f::Function
    #count::Int64
    itpl_point_dict::Dict{Vector{Int}, Float64}
    function WrapFuncFloat(f::Function)::WrapFuncFloat
        new(f, Dict{Vector{Int}, Float64}())
    end
end



function (w::WrapFuncFloat)(bins::Vector{Int})::Float64
    if !haskey(w.itpl_point_dict, bins)
        # FIXED: use deepcopy
        w.itpl_point_dict[deepcopy(bins)] = w.f(bins)
    end
    return w.itpl_point_dict[bins]
end
;

function mps2multivec(mps::TCI.TensorTrain)::Vector{Array{Float64, 3}}
    links = linkinds(mps)
    sites = siteinds(mps)
    N = length(mps)
    res = []
    for i in 1:N
        if i == 1
            push!(res, reshape(array(mps[1], sites[1], links[1]), 1, ITensors.dim(sites[1]), ITensors.dim(links[1])))
        elseif i == N
            push!(res, reshape(array(mps[i], links[end], sites[i]), ITensors.dim(links[end]), ITensors.dim(sites[i]), 1))
        else
            push!(res, array(mps[i], links[i-1], sites[i], links[i]))
        end
    end
    return res
end

function calc_itpl_point_errs(itpl_point_dict, fdata)
    res = 0
    for (key, value) in itpl_point_dict
        idx = Q.quantics_to_index(key)[1]
        res += abs(fdata[idx] - value)
    end
    return res
end


calc_itpl_point_errs (generic function with 1 method)

In [9]:
# 各資産ともグリッドの最大値を使う場合
localindex_v = [100 for _ in 1:2d]

mean_pv, std_pv = calculate_option_price(T, r, K, d, S0, stock0, localindex_v, nPath, σs, corrMat)
println("Present Value: ", mean_pv)
println("Standard Deviation: ", std_pv)
println("Abs.Err.: ", std_pv / sqrt(nPath))


Present Value: 6.16224477441282
Standard Deviation: 9.312806162122191
Abs.Err.: 2.944967887995743


In [10]:
using Random

d = 5
num_grid = 100

# 誤差を記録する配列
abs_errors = Float64[]

Random.seed!(seed)
for _ in 1:100
    # [σ1, S0_1, σ2, S0_2, ..., σd, S0_d] をランダムに生成
    localindex_v = [rand(1:num_grid) for _ in 1:2d]
    mean_pv, std_pv = calculate_option_price(T, r, K, d, S0, stock0, localindex_v, nPath, σs, corrMat)
    abs_err = std_pv / sqrt(nPath)
    push!(abs_errors, abs_err)
end

println("Mean Abs.Err.: ", mean(abs_errors))
println("Max  Abs.Err.: ", maximum(abs_errors))


Mean Abs.Err.: 1.0704463332803593
Max  Abs.Err.: 2.680704658749252


In [18]:
abo_(localindex_v) = calculate_option_price(T, r, K, d, S0, stock0, localindex_v, nPath, σs, corrMat)[1]
localdims_v = fill(num_nodes, 2d)
Random.seed!(1234)
firstpivot_v = rand(1:num_nodes, 2d)
abo = WrapFuncFloat(abo_) # 
phi = TCI.CachedFunction{Float64}(abo, localdims_v)

phi = TCI.CachedFunction{Float64}(abo, localdims_v)




(::TensorCrossInterpolation.CachedFunction{Float64, UInt128}) (generic function with 3 methods)

In [19]:
# TCIワンショット補間関数
function tci_oneshot(func, d, localdims, firstpivot, tol)
    BLAS.set_num_threads(4)
    for _ in 1:100
        p = TCI.optfirstpivot(func, localdims, firstpivot)
        if abs(func(p)) > abs(func(firstpivot))
            firstpivot = p
        end
    end
    qtt, ranks, errors = TCI.crossinterpolate2(Float64, func, localdims, [firstpivot],
                                                  tolerance=tol, maxiter=1, verbosity=1,
                                                  loginterval=1, pivotsearch=:rook, maxbonddim=1)
    return qtt, errors
end

tci_oneshot (generic function with 1 method)

In [20]:
tol_mc = 2.0
tol = tol_mc

tci_time = @elapsed begin
    tt_tci, errors_asianbarrier = tci_oneshot(phi, d, localdims_v, firstpivot_v, tol)
end

iteration = 1, rank = 1, error= 0.0, maxsamplevalue= 9.316950965038105, nglobalpivot=0


152.138707792

In [21]:
tt_tci ###

(::TensorCrossInterpolation.TensorCI2{Float64}) (generic function with 1 method)

In [ ]:
tt0 = TCI.TensorTrain(tt_tci)
itpl_point_dict = abo.itpl_point_dict
indexsets = sort(collect(keys(itpl_point_dict)))
vals = [itpl_point_dict[i] for i in indexsets]
fit_itr = 100 #
ttfit = TCI.TensorTrainFit{Float64}(indexsets, vals, tt0)
x0::Vector{Float64} = TCI.flatten(tt0)

using Zygote
using OptimKit
#using OptimKit
function loss(x) 
    ttfit(x)
end
optimizer = LBFGS(; maxiter=fit_itr, verbosity=1)
function loss_and_grad(x)
    y, (∇,) = withgradient(loss, x)
    return y, ∇
end
xopt, fs, gs, niter, normgradhistory = optimize(loss_and_grad, x0, optimizer)
;

ttopt = TCI.TensorTrain{Float64, 3}(TCI.to_tensors(ttfit, xopt))
tt_tci = ttopt

In [ ]:
JLD2.@save "tt_tci_opt_$nPath" tt_tci

UndefVarError: UndefVarError: `tt_tci` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [16]:
JLD2.@load "tt_tci_$nPath" tt_tci

KeyError: KeyError: key "tt_tci" not found